In [ ]:
import os
from googleapiclient.discovery import build

In [ ]:
# Id of the video whose comments need to be scrapped off
videoId = "Mk-yWIF_92s"

In [ ]:
from API_KEY import stored_api_key
def commentScraper(vidID):

    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    # api_service_name = "youtube"
    # api_version = "v3"
    API_KEY = stored_api_key

    youtube = build(
        "youtube", "v3", developerKey = API_KEY)

    request = youtube.commentThreads().list(
        part="id, replies, snippet",
        # order="time", #Order can be either time or relevance
        maxResults=100, #By deafult, its value is 20
        videoId=vidID
    )
    response = request.execute()
    comments_list = response['items']

    while response.get('nextPageToken', None):
        request = youtube.commentThreads().list(
            part="id, replies, snippet",
            # order="time", #Order can be either time or relevance
            maxResults=100, #By deafult, its value is 20
            videoId=vidID,
            pageToken=response['nextPageToken'] #to get the comment on the next block
        )
        response = request.execute()
        comments_list.extend(response['items'])
      
    

    return comments_list

#Enter your VideoID here
total_comments = commentScraper(videoId)

In [ ]:
scraped_comments_array = []
for i in range(0, len(total_comments)):
  current_scrapped_comment = []
  current_scrapped_comment.append(total_comments[i]['snippet']['topLevelComment']['snippet']['textDisplay'])
  current_scrapped_comment.append(total_comments[i]['snippet']['topLevelComment']['snippet']['authorDisplayName'])
  current_scrapped_comment.append(total_comments[i]['snippet']['topLevelComment']['snippet']['authorChannelId']['value'])
  current_scrapped_comment.append(total_comments[i]['snippet']['topLevelComment']['snippet']['publishedAt'][:10])
  current_scrapped_comment.append(total_comments[i]['snippet']['topLevelComment']['snippet']['likeCount'])
  scraped_comments_array.append(current_scrapped_comment)

  if total_comments[i].get('replies', None):
    for j in range(0, len(total_comments[i]['replies']['comments'])):
      current_scrapped_comment = []
      current_scrapped_comment.append(total_comments[i]['replies']['comments'][j]['snippet']['textDisplay'])
      current_scrapped_comment.append(total_comments[i]['replies']['comments'][j]['snippet']['authorDisplayName'])
      current_scrapped_comment.append(total_comments[i]['replies']['comments'][j]['snippet']['authorChannelId']['value'])
      current_scrapped_comment.append(total_comments[i]['replies']['comments'][j]['snippet']['publishedAt'][:10])
      current_scrapped_comment.append(total_comments[i]['replies']['comments'][j]['snippet']['likeCount'])
      scraped_comments_array.append(current_scrapped_comment)

In [ ]:
# importing pandas as pd  
import pandas as pd  
  
     
# list of comment, name, channelId, date, like
comment = []
name = []
channelId = []
date = []
like = []

for i in range(0, len(scraped_comments_array)):
  comment.append(scraped_comments_array[i][0])
  name.append(scraped_comments_array[i][1])
  channelId.append(scraped_comments_array[i][2])
  date.append(scraped_comments_array[i][3])
  like.append(scraped_comments_array[i][4])
     
# dictionary of lists  
dict = {'Comment':comment, 'AuthorName':name, 'AuthorChannelId':channelId, 'PublishedDate':date, 'LikeCount':like}  
df = pd.DataFrame(dict) 

# saving the dataframe 
df.to_csv('YoutubeScrapped.csv')